In [1]:
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd

## Extract reviews on trustpilot
This part aims at extract reviews of companys on the website trust pilot. Function `exctract_info` is designed to extract reviews and notes of this website.

In [ ]:
for i in range(len(notes)):
        note = pattern_note.findall(notes_soup[i].find("img")['alt'])
        
        if len(pattern_note.findall(notes_soup[i].find("img")['alt'])) == 1:
            notes[i] = int(note[0])

In [12]:
def extract_info_TP(url) :

    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    reviews_soup = soup.find_all("p", class_="typography_body-l__KUYFJ typography_appearance-default__AAY17 typography_color-black__5LYEn")
    reviews = [reviews_soup[i].get_text() for i in range(len(reviews_soup))]

    titles_soup = soup.find_all("h2", class_="typography_heading-s__f7029 typography_appearance-default__AAY17")
    titles = [titles_soup[i].get_text() for i in range(len(titles_soup))]
  

    notes = np.zeros(len(reviews))
    notes_soup = soup.find_all("div", class_="star-rating_starRating__4rrcf star-rating_medium__iN6Ty")
    pattern_note = re.compile("Rated (.) out of 5")
    for i in range(len(notes)):
        note = pattern_note.findall(notes_soup[i].find("img")['alt'])
        if len(pattern_note.findall(notes_soup[i].find("img")['alt'])) == 1:
            notes[i] = int(note[0])
    
    return pd.DataFrame({'Notes': notes, 'title' : titles,'reviews':reviews})

In [11]:
pattern_note = re.compile("Rated (.) out of 5")
int(pattern_note.findall("Rated 1 out of 5")[0])

1

In [23]:
url_start = 'https://www.trustpilot.com/review/www.aircanada.com'
df = extract_info_TP(url_start)
nb_pages = 7
for i in np.arange(1,nb_pages+1):
    url = 'https://www.trustpilot.com/review/www.aircanada.com?page='+str(i)
    df = pd.concat([df, extract_info_TP(url)])


In [24]:
df = df[df.Notes <3 ]

In [25]:
df.to_csv("air_canada_reviews.csv", columns = ["Notes", "title", "reviews"])

,Notes,title,reviews
0,0.0,Terrible experience with the maple…,Terrible experience with the maple lounge who ...
1,1.0,Delayed by 12hours on flight into…,Delayed by 12hours on flight into Toronto from...
2,1.0,Air Canada Hates Handicapped Passengers,Summary: A Third-World Airline run by a bunch ...
3,1.0,Worst airline imaginable,Connecting flight delayed 24 hours. Stuck in f...
4,1.0,Never Again,When you think of a destination wedding in tan...
...,...,...,...
15,1.0,Be warned. A very dodgy company.,Be warned. A very dodgy company. They lost 4...
16,1.0,I had read bad reviews of this company,"I had read bad reviews of this company, but it..."
17,2.0,I hate Air Canada,I paid several thousand dollars for a flight f...
18,1.0,Reserved seat changed so party split up.,"Paid to reserve seats together, but changed wi..."


In [79]:
def extract_info_airlines_eq(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")

    date_soup = soup.find_all("h3", class_="text_sub_header userStatusWrapper")
    dates = [date_soup[i].find("time")['datetime'] for i in range(len(date_soup))]  
 
    reviews_soup = soup.find_all("div", class_="text_content")
    reviews = ['a']*len(reviews_soup)
    rev = re.compile(' (Verified |) (.*)') #To delete verified from the comment

    for i in range(len(reviews_soup)):
        reviews_get = rev.findall(reviews_soup[i].get_text())
        if len(reviews_get) == 0 : 
            reviews[i]= reviews_soup[i].get_text()
        else:
            reviews[i] = reviews_get[0][1]
                        
    titles_soup = soup.find_all("h2", class_="text_header")
    titles = [titles_soup[i].get_text() for i in range(len(dates))]  

    notes_soup = soup.find_all("div", class_="rating-10")
    notes = [int(notes_soup[i].find("span", {'itemprop': 'ratingValue'}).text) for i in range(1,len(notes_soup))]

    return pd.DataFrame({'Notes': notes, 'title' : titles,'reviews':reviews, 'Dates':pd.to_datetime(dates)})

In [55]:
a = ['a',]*5
a

['a', 'a', 'a', 'a', 'a']

In [80]:
url = 'https://www.airlinequality.com/airline-reviews/air-transat/'
df = extract_info_airlines_eq(url)

In [81]:
df

,Notes,title,reviews,Dates
0,1,"""short of my expectations""","""I would like to express my deep dissatisfacti...",2023-10-18
1,1,"""The airline is dysfunctional""","The airline is dysfunctional, from check in to...",2023-10-12
2,2,"""could not print me a boarding pass","If you book a codeshare flight, there is no w...",2023-10-08
3,10,"""his dedication and hard work""",We are very grateful and thankful to all the ...,2023-10-05
4,4,"""dissatisfied and disappointed""",✅ Trip Verified | Sadly I do not recommend thi...,2023-09-24
5,4,"""All in all never again""",Our Air Transat service (with its own logos on...,2023-09-17
6,2,"""Horrible seats""",✅ Trip Verified | Horrible seats. too small. S...,2023-09-12
7,1,"""Terrible service""",This aircraft doesn't have any charging ports ...,2023-09-11
8,2,"""very disappointing experience""",We booked club class in order to ensure a good...,2023-09-06
9,1,"""I’ve spent hours on the phone""","I am not travelling until March 2024, but just...",2023-08-27


In [85]:
df['reviews'][4]

'✅ Trip Verified | Sadly I do not recommend this airline that boasts itself as being voted best leisure airline of the year by SKYTRAX! As I have flown this route before a few years ago and again this March down south to Mexico their quality of service has downgraded a lot! The staff is unhappy or so or seems the on board service is that which resembles carriers like Frontier and Spirit. They no longer offer the option of pre ordering a gourmet meal since the food for this route is minuscule and awful! The seats are uncomfortable and the aisles are so narrow I don’t know how their flight attendants can get through especially in an emergency. The only thing going for them is the monopoly of a direct flight from Montreal to Athens alongside Air Canada. The language fluency in French was horrendous as the head flight attendants struggled both in French and especially Greek like they were reading off cue cards. Canada is a bilingual country and therefore the staff on board should be biling

In [90]:
rev = re.compile('(.*) (|) (.*) ')
rev.findall(df['reviews'][4])

[]